In [1]:
import pandas as pd
import pyodbc

In [2]:
sql = """
--Active_Parts_List
 SELECT
    PartDet.PartID,
    PartDet.BasePartNumber,
    PartDet.PartName,
    SupInfo.SupNum,
    SupInfo.SupplierName,
    ModInfo.ModelYear,
    ModInfo.VehicleModelCode,
    EvInfo.EventName,
    PartDet.NMMP
FROM
    (
    SELECT
        DISTINCT dbo.Stg_Part_Details_TBL.PartID,
        dbo.Stg_Part_Details_TBL.BasePartNumber,
        dbo.Stg_Part_Details_TBL.PartName,
        dbo.Stg_Part_Details_TBL.EventID,
        dbo.Stg_Part_Details_TBL.ModelID,
        dbo.Stg_Part_Details_TBL.OEMSupplierID,
        'MP' as NMMP
    FROM
        dbo.Stg_Part_Details_TBL
    LEFT JOIN dbo.Stg_Events_TBL ON
        dbo.Stg_Part_Details_TBL.EventID = dbo.Stg_Events_TBL.EventID
    WHERE
        dbo.Stg_Part_Details_TBL.IsPartActive = 1
        AND dbo.Stg_Part_Details_TBL.IsInterCompanyPart = 0
        AND dbo.Stg_Part_Details_TBL.PartType = 'NORMAL'
        AND dbo.Stg_Events_TBL.EventMassProductionDate < GETDATE()
UNION
    SELECT
        DISTINCT dbo.Stg_Part_Details_TBL.PartID,
        dbo.Stg_Part_Details_TBL.BasePartNumber,
        dbo.Stg_Part_Details_TBL.PartName,
        dbo.Stg_Part_Details_TBL.EventID,
        dbo.Stg_Part_Details_TBL.ModelID,
        dbo.Stg_Part_Details_TBL.OEMSupplierID,
        'NM' as NMMP
    FROM
        dbo.Stg_Part_Details_TBL
    LEFT JOIN dbo.Stg_Events_TBL ON
        dbo.Stg_Part_Details_TBL.EventID = dbo.Stg_Events_TBL.EventID
    WHERE
        dbo.Stg_Part_Details_TBL.IsPartActive = 1
        AND dbo.Stg_Part_Details_TBL.IsInterCompanyPart = 0
        AND dbo.Stg_Part_Details_TBL.PartType = 'NORMAL'
        AND dbo.Stg_Events_TBL.EventMassProductionDate >= GETDATE() ) PartDet
Right Join (
    SELECT
        ModelID,
        ModelYear,
        TargetPlantLocationCode,
        VehicleID,
        VehicleModelCode,
        EventID
    FROM
        Stg_ModelMasterDetails_TBL
    WHERE
        TargetPlantLocationCode = 'H' ) ModInfo ON
    PartDet.ModelID = ModInfo.ModelID
Right Join (
    SELECT
        EventID,
        EventName
    FROM
        dbo.Stg_Events_TBL
    WHERE
        PlantID = 8
        AND IsEventActive = 1) EvInfo ON
    PartDet.EventID = EvInfo.EventID
Left Join (
    SELECT
        Concat(SupplierNumber, SupplierLocCode) AS SupNum,
        SupplierName,
        SupplierID
    FROM
        dbo.Stg_SupplierMaster_TBL) SupInfo ON
    PartDet.OEMSupplierID = SupInfo.SupplierID
Where
    PartID Is Not Null
"""

#### Usually best practice to use context manager ("with" statement) so that we don't need to worry about closing database connections

Review this [page](https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-SQL-Server-from-Windows) for connection string settings for both DSN and DSN-less (hostname) connection strings

In [3]:
with pyodbc.connect('Driver={SQL Server};'
                    'Server=YOUR_SERVER;'
                    'Database=NAPS_Reporting;'
                    'Trusted_Connection=yes;') as conn:
    df = pd.read_sql(sql, conn)

In [4]:
df.head()

,PartID,BasePartNumber,PartName,SupNum,SupplierName,ModelYear,VehicleModelCode,EventName,NMMP
0,1754968,31500TBA A100M1,"BATTERY ASSY, L2 (FLA)",10038001,"CLARIOS, LLC",2020.0,4DR CIVIC,2020 Civic_MP,MP
1,1645398,39120T2A A000M1,"SPEAKER ASSY,TWEETER",16102012,PIONEER AUTOMOTIVE TECHNOLOGIES INC,2021.0,CRV FHEV,2020 CRV 2WD_MP,MP
2,1645399,39120T2A A000M1,"SPEAKER ASSY,TWEETER",16102012,PIONEER AUTOMOTIVE TECHNOLOGIES INC,2021.0,CRV FHEV,2020 CRV 4WD_MP,MP
3,1646094,39186TLA A000M1,"UNIT ASSY,AUDIO AMP(HIGH POWER",16102012,PIONEER AUTOMOTIVE TECHNOLOGIES INC,2021.0,CRV FHEV,2020 CRV 2WD_MP,MP
4,1646095,39186TLA A000M1,"UNIT ASSY,AUDIO AMP(HIGH POWER",16102012,PIONEER AUTOMOTIVE TECHNOLOGIES INC,2021.0,CRV FHEV,2020 CRV 4WD_MP,MP
